In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('http://www.reuters.com/robots.txt')
rp.read()

In [4]:
rp.can_fetch(useragent='*',url='http://reuters.com/sitemap.xml')

In [8]:
sitemap = rp.site_maps()
sitemap

In [9]:
!pip install xmltodict

In [22]:
import xmltodict
import requests

url = sitemap[0] 
sitemaps = xmltodict.parse(requests.get(url).text)
sitemaps


In [43]:
#sitemap의 url을 확보
#먼저 key값 확인 -> .keys()

sub_sitemaps = [sitemap['loc'] for sitemap in sitemaps['sitemapindex']['sitemap']]

In [ ]:
response = requests.get(sub_sitemaps[0])
urls = xmltodict.parse(response.text)
urls

In [67]:
urls['urlset']['url']

In [87]:
#sub_sitemaps = [sitemap['loc'] for sitemap in sitemaps['sitemapindex']['sitemap']]
news_urls = [url['loc'] for url in urls['urlset']['url']]
news_urls

['https://www.reuters.com/markets/europe/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28/',
 'https://www.reuters.com/business/finance/oaktree-capital-seeks-raise-10-billion-new-fund-ft-2023-02-28/',
 'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/',
 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28/',
 'https://www.reuters.com/markets/asia/indias-adani-plans-repay-up-790-mln-share-backed-loans-by-march-sources-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/smith-says-australia-batsmen-will-ditch-risky-tempo-third-test-2023-02-28/',
 'https://www.reuters.com/technology/cryptoverse-bitcoin-moves-towards-satoshis-payment-dream-2023-02-28/',
 'https://www.reuters.com/markets/europe/ecb-has-started-win-inflation-fight-lane-says-2023-02-28/',
 'https://www.reuters.com/markets/ra

In [73]:
%%time

#데이터 다운로드
#session 추상화
#%%time -> 실행하는데 걸리는 시간 / 맨 위에 작성


session = requests.Session()

for url in news_urls[:5]:
    file = url.split('/')[-2] + '.html'

    response = session.get(url)
    if response.ok:
        with open('datas/'+file, 'w+b') as f:
            f.write(response.text.encode('utf-8'))
    else:
        print(f'error widh URL : {url}')

Wall time: 5.07 s


In [80]:
#filename.txt 파일을 쓰기 모드(w)로 열기. open()함수는 파일 객체 변환
file = open('filename.txt','w')

#파일 객체의 write()메서드를 통해 문자열을 파일에 쓸 수 있습니다.
file.write('파일에 작성할 문자열')

#파일에 대한 처리가 끝났다면 반드시 파일 객체를 닫아줘야 합니다.

file.close()

In [78]:
#close에 대한 처리 없이 간편히 쓰려면 with 활용
#아래 코드는 이전에 작성했던 것과 동일한 동작을 수행
with open('filename.txt','w')as file:
    file.write('파일에 작성할 문자열')
    

In [81]:
with open('filename.txt','r') as file:
    print(file.read())
    

파일에 작성할 문자열


In [83]:
import os
path = './datas/'
files = [path + file for file in os.listdir(path)]
files

['./datas/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28.html',
 './datas/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28.html',
 './datas/global-markets-view-europe-2023-02-28.html',
 './datas/home-zara-fast-slow-fashion-collide-2023-02-28.html',
 './datas/oaktree-capital-seeks-raise-10-billion-new-fund-ft-2023-02-28.html']

In [85]:
with open(files[2],'r',encoding = 'utf-8') as f:
    html = f.read()

In [90]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'html.parser')
title = soup.select_one('h1')
print(title.text)

Morning Bid: Market mood downbeat ahead of raft of data


In [89]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'html.parser')
title = soup.select_one('#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > header > div > div > h1')
print(title.text)


Morning Bid: Market mood downbeat ahead of raft of data


In [94]:
soup.h1

<h1 class="text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_3__1kDhc heading__base__2T28j heading__heading_3__3aL54 article-header__title__3Y2hh" data-testid="Heading">Morning Bid: Market mood downbeat ahead of raft of data</h1>

In [92]:
soup.title.text

'Morning Bid: Market mood downbeat ahead of raft of data | Reuters'

In [102]:
soup.select_one('#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > div > div > div.article-body__content__17Yit.paywall-article > p:nth-child(2)').text

#soup.select('p') -> p의 값을 가지는 모든 데이터를 리스트로 저장 

'A look at the day ahead in European and global markets from Anshuman Daga'

In [96]:
soup.find('link',{'rel':'canonical'})['href']

'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/'